In [1]:
from tqdm import tqdm
import random
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [5]:
def diameter_computation(G,d):
    #Check for giant connected component
    giant = G
    if not nx.is_connected(G):
        Gcc   = sorted(nx.connected_components(G), key=len, reverse=True)
        giant = G.subgraph(Gcc[0])
    
    # finding the best L(double_sweep) and U(fringe)
    f_bfs  = 0
    S_U = 0
    S_L = 0
    L = 0
    U = len(giant.nodes())
    
    for i in tqdm(range(d)):
        l,r = double_sweep(giant)
        S_L += 2
        if l>L:
            L = l
        u,B = fringe(giant,r,U)
        S_U += B
        if u<U:
            U=u
    S_L /= d
    S_U /= d
    
     # Diameter calculation
    if U == L:
        print('Fringe converged')
        print('The diameter is: ', L)
        print('Number of BFSes needed:',S_L*d,'from double sweep, and',S_U*d,'from fringe')
        print('The total number of BFSes required for double sweep devided with the number of iterations is:',S_L)
        print('The total number of BFSes required for fringe devided with the number of iterations is:',S_U)
    else:
        print('Fringe did not converged')
        print('The diameter is between L = ',L, 'and U = ',U)
        print('The approximated diameter is: ', L)
        print('Number of BFSes needed:',S_L*d,'from double sweep, and',SU*d,'from fringe')
        print('The total number of BFSes required for double sweep devided with the number of iterations is:',S_L)
        print('The total number of BFSes required for fringe devided with the number of iterations is:',S_U)
    return

In [6]:
def double_sweep(giant):
    
    #random node from nodes in giant component
    u = random.choice(list(giant.nodes))
    ecc_u = nx.eccentricity(giant,u)
    a = list(nx.descendants_at_distance(giant,u,ecc_u))[0]
    ecc_a = nx.eccentricity(giant,a)
    
    L = ecc_a

    # r is the node in the middle of path a-b.
    l = round(ecc_a/2)

    # is r really the middle of a-b? 
    r = random.choice(list(nx.descendants_at_distance(giant,a,l)))

    return L,r

In [7]:
def fringe(giant,r,U):
    
    f_max = 1000
    
    # We get the fringe of r.
    ecc_r = nx.eccentricity(giant,r)
    
    # If 2*ecc_r-2 isn't an improvement of U (calculated in the previous iteration) we stop the procedure
    if (2*ecc_r-2) >= U:
        bfs = 0
        pass
    else:
        F = list(nx.descendants_at_distance(giant,r,ecc_r))

        #Calculate the maximum eccentricity of the nodes in the fringe of r.
        B = 0

        if len(F) < f_max:
            bfs = len(F)
            for i in F:
                ecc_i = nx.eccentricity(giant,i)
                if ecc_i > B:
                    B = ecc_i

            # Calculating the upper bound
            if len(F)>1 and B == 2 * ecc_r -1:
                U = 2 * ecc_r - 1
            elif len(F)>1 and B<2*ecc_r-1:
                U = 2*ecc_r -2
            else:
                U = B

        #If |F| > F_max then U = diam(T_r) for which we choose
        #the eccentricity of a random node in the fringe of u.
        else:
            B = nx.eccentricity(giant,random.choice(F))
            U = 2*ecc_r

    return U,bfs

In [15]:
#Diameter = 15 (CORRECT)
data = open(r"C:\Users\geo_l\OneDrive\Υπολογιστής\Courses\1st Semester\SNACS\project\data\facebook_large\facebook_large\musae_facebook_edges.csv",
            'r',encoding='utf8').readlines()
data = [i.strip('\n') for i in data]
data = data[1:]\

nodes = [i for i in range(22470)]

edges = []

for i in data:
    edges.append((int(i.split(',')[0]),int(i.split(',')[1])))
    
F = nx.Graph()
F.add_nodes_from(nodes)
F.add_edges_from(edges)

In [30]:
diameter_computation(F,10)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.23it/s]

Fringe converged
The diameter is:  15
Number of BFSes needed: 2 from double sweep, and 20 from fringe
The total number of BFSes required for double sweep devided with the number of iterations is: 2.0
The total number of BFSes required for fringe devided with the number of iterations is: 2.0


In [5]:
#Diameter = 7 (CORRECT)
data = open(r"C:\Users\geo_l\OneDrive\Υπολογιστής\Courses\1st Semester\SNACS\project\data\wiki-Vote.txt\wiki-Vote.txt",
            'r',encoding='utf8').readlines()
data = data[4:]
data = [i.strip('\n') for i in data]
data = [i.split('\t') for i in data]

nodes = []
edges = []

for i in data:
    edges.append((int(i[0]),int(i[1])))
    for j in range(2):
        cur_num = int(i[j])
        if cur_num not in nodes:
            nodes.append(cur_num)

W = nx.Graph()
W.add_nodes_from(nodes)
W.add_edges_from(edges)

In [14]:
diameter_computation(W,10)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:49<00:00,  4.96s/it]

Fringe converged
The diameter is:  7
Number of BFSes needed: 2 from double sweep, and 89 from fringe
The total number of BFSes required for double sweep devided with the number of iterations is: 2.0
The total number of BFSes required for fringe devided with the number of iterations is: 8.9


In [20]:
#Diameter = 15 (CORRECT)
data = open(r'C:\Users\geo_l\OneDrive/Υπολογιστής/Courses/1st Semester/SNACS/project/data/soc-Epinions1.txt/soc-Epinions1.txt',encoding='utf8').readlines()
data =  data[4:]
data = [i.strip('\n') for i in data]
data = [i.split('\t') for i in data]

#noded
nodes= data
nodes = [j for i in nodes for j in i]
nodes = list(set(nodes))

#edges
edges = data
for i in range(len(edges)):
    data[i] = (int(edges[i][0]),int(edges[i][1]))
    
E = nx.Graph()
E.add_nodes_from(nodes)
E.add_edges_from(edges)

In [21]:
diameter_computation(E,10)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:05<00:00, 12.54s/it]

Fringe converged
The diameter is:  15
Number of BFSes needed: 2 from double sweep, and 18 from fringe
The total number of BFSes required for double sweep devided with the number of iterations is: 2.0
The total number of BFSes required for fringe devided with the number of iterations is: 1.8


In [32]:
#Diameter = 9 (CORRECT)
data = open(r"C:\Users\geo_l\OneDrive\Υπολογιστής\Courses\1st Semester\SNACS\project\data\soc-sign-bitcoinotc.csv\soc-sign-bitcoinotc.csv").readlines()
data = [i.split(',') for i in data]
edges = []
for i in range(len(data)):
    edges.append((int(data[i][0]),int(data[i][1])))
nodes = []
for i in edges:
    for j in i:
        nodes.append(j)
nodes = list(set(nodes))
B = nx.Graph()
B.add_nodes_from(nodes)
B.add_edges_from(edges)

In [50]:
diameter_computation(B,10)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:10<00:00,  1.01s/it]

Fringe converged
The diameter is:  9
Number of BFSes needed: 20.0 from double sweep, and 35 from fringe
The total number of BFSes required for double sweep devided with the number of iterations is: 2.0
The total number of BFSes required for fringe devided with the number of iterations is: 4.1


In [51]:
# Diameter = 12 (CORRECT)
data = open(r"C:\Users\geo_l\OneDrive\Υπολογιστής\Courses\1st Semester\SNACS\project\data\soc-Slashdot0811.txt\soc-Slashdot0811.txt").readlines()
data = data[4:]
data = [i.strip("\n") for i in data]
data = [i.split("\t") for i in data]
nodes= data
nodes = [j for i in nodes for j in i]
nodes = list(set(nodes))
edges = data
for i in range(len(edges)):
    data[i] = (int(edges[i][0]),int(edges[i][1])) 
S = nx.Graph()
S.add_nodes_from(nodes)
S.add_edges_from(edges)

In [52]:
diameter_computation(S,10)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:05<00:00, 12.59s/it]

Fringe converged
The diameter is:  12
Number of BFSes needed: 20.0 from double sweep, and 3 from fringe
The total number of BFSes required for double sweep devided with the number of iterations is: 2.0
The total number of BFSes required for fringe devided with the number of iterations is: 0.3


In [53]:
edges = open(r"C:\Users\geo_l\OneDrive\Υπολογιστής\Courses\1st Semester\SNACS\project\data\git_web_ml\git_web_ml\musae_git_edges.csv").readlines()
edges = edges[1:]
edges = [i.strip("\n") for i in edges]
edges = [i.split(',') for i in edges]
for i in range(len(edges)):
    edges[i] = (int(edges[i][0]),int(edges[i][1]))
    
nodes =open(r"C:\Users\geo_l\OneDrive\Υπολογιστής\Courses\1st Semester\SNACS\project\data\git_web_ml\git_web_ml\musae_git_target.csv").readlines()
nodes = nodes[1:]
nodes = [i.split(',') for i in nodes]
nodes = [int(i[0]) for i in nodes]

Git = nx.Graph()
Git.add_nodes_from(nodes)
Git.add_edges_from(edges)

In [94]:
diameter_computation(Git,10)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:14<00:00,  1.44s/it]

Fringe converged
The diameter is:  11
Number of BFSes needed: 20.0 from double sweep, and 9 from fringe
The total number of BFSes required for double sweep devided with the number of iterations is: 2.0
The total number of BFSes required for fringe devided with the number of iterations is: 1.5


In [4]:
import gzip
import shutil
# with gzip.open(r"C:\Users\geo_l\OneDrive\Υπολογιστής\Courses\1st Semester\SNACS\project\data\facebook_combined.txt.gz", 'rb') as f_in:
#     with open(r"C:\Users\geo_l\OneDrive\Υπολογιστής\Courses\1st Semester\SNACS\project\data\facebook_combined.txt", 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)
data = open(r"C:\Users\geo_l\OneDrive\Υπολογιστής\Courses\1st Semester\SNACS\project\data\facebook_combined.txt").readlines()
data = [i.strip("\n") for i in data]
data = [i.split(" ") for i in data]
nodes = data; edges = data
nodes = [j for i in nodes for j in i]
nodes = list(set(nodes))
for i in edges:
    i = (int(i[0]),int(i[1]))
FC = nx.Graph()
FC.add_nodes_from(nodes)
FC.add_edges_from(edges)

In [8]:
diameter_computation(FC,10)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.23it/s]

Fringe converged
The diameter is:  8
Number of BFSes needed: 20.0 from double sweep, and 197.0 from fringe
The total number of BFSes required for double sweep devided with the number of iterations is: 2.0
The total number of BFSes required for fringe devided with the number of iterations is: 19.7


In [7]:
with gzip.open(r"C:\Users\geo_l\OneDrive\Υπολογιστής\Courses\1st Semester\SNACS\project\data\roadNet-CA.txt.gz", 'rb') as f_i:
    with open(r"C:\Users\geo_l\OneDrive\Υπολογιστής\Courses\1st Semester\SNACS\project\data\roadNet-CA.txt", 'wb') as f_o:
        shutil.copyfileobj(f_i, f_o)
data = open(r"C:\Users\geo_l\OneDrive\Υπολογιστής\Courses\1st Semester\SNACS\project\data\roadNet-CA.txt").readlines()
data = data[4:]
data = [i.strip("\n") for i in data]
data = [i.split("\t") for i in data]
nodes = data; edges = data
nodes = [j for i in nodes for j in i]
nodes = list(set(nodes))
for i in edges:
    i = (int(i[0]),int(i[1]))
CA = nx.Graph()
CA.add_nodes_from(nodes)
CA.add_edges_from(edges)

In [12]:
diameter_computation(CA,3)

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [17:30<00:00, 350.10s/it]


Fringe did not converged
The diameter is between L =  855 and U =  853
The approximated diameter is:  855
Number of BFSes needed: 6.0 from double sweep, and 9 from fringe
The total number of BFSes required for double sweep devided with the number of iterations is: 2.0
The total number of BFSes required for fringe devided with the number of iterations is: 3.0


In [2]:
data = open(r"C:\Users\geo_l\OneDrive\Υπολογιστής\Courses\1st Semester\SNACS\project\data\com-youtube.ungraph.txt").readlines()
data = [i.strip("\n") for i in data]
data = [i.split("\t") for i in data]
nodes = data; edges = data
nodes = [j for i in nodes for j in i]
nodes = list(set(nodes))
for i in edges:
    i = (int(i[0]),int(i[1]))
YT = nx.Graph()
YT.add_nodes_from(nodes)
YT.add_edges_from(edges)

In [45]:
diameter_computation(YT,10)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [05:29<00:00, 32.90s/it]

Fringe converged
The diameter is:  24
Number of BFSes needed: 20.0 from double sweep, and 2.0 from fringe
The total number of BFSes required for double sweep devided with the number of iterations is: 2.0
The total number of BFSes required for fringe devided with the number of iterations is: 0.2


In [34]:
with gzip.open(r"C:\Users\geo_l\OneDrive\Υπολογιστής\Courses\1st Semester\SNACS\project\data\email-Eu-core.txt.gz", 'rb') as f_in:
    with open(r"C:\Users\geo_l\OneDrive\Υπολογιστής\Courses\1st Semester\SNACS\project\data\email-Eu-core.txt", 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
data = open(r"C:\Users\geo_l\OneDrive\Υπολογιστής\Courses\1st Semester\SNACS\project\data\email-Eu-core.txt").readlines()

In [37]:
data = [i.strip("\n") for i in data]
data = [i.split(" ") for i in data]
nodes = data; edges = data
nodes = [j for i in nodes for j in i]
nodes = list(set(nodes))
for i in edges:
    i = (int(i[0]),int(i[1]))
EU = nx.Graph()
EU.add_nodes_from(nodes)
EU.add_edges_from(edges)

In [44]:
diameter_computation(EU,10)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.13it/s]

Fringe converged
The diameter is:  7
Number of BFSes needed: 20.0 from double sweep, and 120.0 from fringe
The total number of BFSes required for double sweep devided with the number of iterations is: 2.0
The total number of BFSes required for fringe devided with the number of iterations is: 12.0
